In [26]:
import requests
import pandas as pd
import numpy as np
import random

from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize

from IPython.display import Image 
from IPython.core.display import HTML 

#!conda install -c conda-forge folium=0.5.0 --yes
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

### Parsing city-data.com for schools names and rating

In [2]:
#URL of the web page that has all schools rating.
url = 'http://www.city-data.com/schools-dirs/schools-NC.html'

In [3]:
#Parse the 7th table on the web page and convert "-" values in Rationg to NaN.
table = pd.read_html(url, na_values = ['-'])[7]
table.head(7)

,Name,Address,Rating,Grade span
0,A G Cox Middle,"2657 Church Street, Winterville",50.0,6-8
1,A H Snipes Academy of Arts/Des,"2150 Chestnut Street, Kings Grant",9.0,KG-5
2,A L Brown High,"415 E 1st Street, Concord",59.0,9-12
3,A L Stanback Middle,"3700 NC 86 South, Hillsborough",69.0,6-8
4,A T Allen Elementary,"3939 Abilene Rd, Concord",69.0,KG-5
5,Aberdeen Elementary,"503 Sandhills Blvd N, Aberdeen",21.0,3-5
6,Aberdeen Primary,"310 Keyser St, Aberdeen",NaN,PK-2


In [4]:
#The city-data.com has other 25 pages listing the schools. We create a loop to parse all and save them.
url_core = 'http://www.city-data.com/schools-dirs/schools-NC'
i = 1
while (i < 26):
    url_pages = url_core + str(i) + '.html'
    table = table.append(pd.read_html(url_pages, na_values = ['-'])[7])
    i = i + 1    
    print(url_pages)
    

table.shape

http://www.city-data.com/schools-dirs/schools-NC1.html
http://www.city-data.com/schools-dirs/schools-NC2.html
http://www.city-data.com/schools-dirs/schools-NC3.html
http://www.city-data.com/schools-dirs/schools-NC4.html
http://www.city-data.com/schools-dirs/schools-NC5.html
http://www.city-data.com/schools-dirs/schools-NC6.html
http://www.city-data.com/schools-dirs/schools-NC7.html
http://www.city-data.com/schools-dirs/schools-NC8.html
http://www.city-data.com/schools-dirs/schools-NC9.html
http://www.city-data.com/schools-dirs/schools-NC10.html
http://www.city-data.com/schools-dirs/schools-NC11.html
http://www.city-data.com/schools-dirs/schools-NC12.html
http://www.city-data.com/schools-dirs/schools-NC13.html
http://www.city-data.com/schools-dirs/schools-NC14.html
http://www.city-data.com/schools-dirs/schools-NC15.html
http://www.city-data.com/schools-dirs/schools-NC16.html
http://www.city-data.com/schools-dirs/schools-NC17.html
http://www.city-data.com/schools-dirs/schools-NC18.html
h

(2590, 4)

In [5]:
#Remove rows with Rating as NaN.
table.dropna(subset=['Rating'], axis=0, inplace=True)
table.head()

,Name,Address,Rating,Grade span
0,A G Cox Middle,"2657 Church Street, Winterville",50.0,6-8
1,A H Snipes Academy of Arts/Des,"2150 Chestnut Street, Kings Grant",9.0,KG-5
2,A L Brown High,"415 E 1st Street, Concord",59.0,9-12
3,A L Stanback Middle,"3700 NC 86 South, Hillsborough",69.0,6-8
4,A T Allen Elementary,"3939 Abilene Rd, Concord",69.0,KG-5


In [6]:
table.shape

(2448, 4)

In [7]:
#Remove schools outside Durham area.
table_Durham = table[table['Address'].str.contains("Durham")]
table_Durham.head()

,Name,Address,Rating,Grade span
66,Bethesda Elementary,"2009 S Miami Boulevard, Durham",18.0,KG-5
25,Brogden Middle,"1001 Leon Street, Durham",37.0,6-8
54,Burton Elementary,"1500 Mathison Avenue, Durham",21.0,KG-5
12,Carter Community Charter,"4100 N. Roxboro Road, Durham",22.0,KG-8
61,Central Park School For Child,"724 Foster Street, Durham",74.0,KG-5


In [8]:
table_Durham.shape

(41, 4)

In [9]:
#Add ', NC' to the Address.
table_Durham['Address'] = table_Durham['Address'].astype(str) + ', NC'
table_Durham.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Name,Address,Rating,Grade span
66,Bethesda Elementary,"2009 S Miami Boulevard, Durham, NC",18.0,KG-5
25,Brogden Middle,"1001 Leon Street, Durham, NC",37.0,6-8
54,Burton Elementary,"1500 Mathison Avenue, Durham, NC",21.0,KG-5
12,Carter Community Charter,"4100 N. Roxboro Road, Durham, NC",22.0,KG-8
61,Central Park School For Child,"724 Foster Street, Durham, NC",74.0,KG-5


In [10]:
#Define the geolocator and get the coordinates of city of Durham.
address = 'Durham City, NC'

geolocator = Nominatim(user_agent="Durham_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Durham are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Durham are 36.0181316, -78.8751582.


In [14]:
#Reset index
table_Durham.reset_index(drop = True, inplace = True)

#Get schools coordinates and add them as two columns to the table.
table_Durham['Latitude'] = ''
table_Durham['Longitude'] = ''


i = 0
for i in range (0,41):
    try:
        location = geolocator.geocode(table_Durham['Address'][i])
        table_Durham['Latitude'][i] = location.latitude
        table_Durham['Longitude'][i] = location.longitude
    except:
        table_Durham['Latitude'][i] = 'NaN'
        table_Durham['Longitude'][i] = 'NaN'
        
table_Durham

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://panda

,Name,Address,Rating,Grade span,Latitude,Longitude
0,Bethesda Elementary,"2009 S Miami Boulevard, Durham, NC",18.0,KG-5,35.9407,-78.8378
1,Brogden Middle,"1001 Leon Street, Durham, NC",37.0,6-8,36.0245,-78.9093
2,Burton Elementary,"1500 Mathison Avenue, Durham, NC",21.0,KG-5,NaN,NaN
3,Carter Community Charter,"4100 N. Roxboro Road, Durham, NC",22.0,KG-8,NaN,NaN
4,Central Park School For Child,"724 Foster Street, Durham, NC",74.0,KG-5,36.0046,-78.901
5,Chewning Middle,"5001 Red Mill Road, Durham, NC",21.0,8,36.0717,-78.8235
6,City of Medicine Academy,"301 Crutchfield Road, Durham, NC",37.0,9-12,NaN,NaN
7,Club Boulevard Elementary,"400 W Club Boulevard, Durham, NC",22.0,KG-5,36.0192,-78.8966
8,Durham School of the Arts,"401 North Duke Street, Durham, NC",75.0,6-12,36.0025,-78.9064
9,Durham's Performance Learning Center,"401 North Driver Street, Durham, NC",8.0,8-12,35.9896,-78.8795


In [15]:
#Remove rows with coordinates as NaN.
table_Durham = table_Durham[table_Durham.Latitude !='NaN']
table_Durham

,Name,Address,Rating,Grade span,Latitude,Longitude
0,Bethesda Elementary,"2009 S Miami Boulevard, Durham, NC",18.0,KG-5,35.9407,-78.8378
1,Brogden Middle,"1001 Leon Street, Durham, NC",37.0,6-8,36.0245,-78.9093
4,Central Park School For Child,"724 Foster Street, Durham, NC",74.0,KG-5,36.0046,-78.901
5,Chewning Middle,"5001 Red Mill Road, Durham, NC",21.0,8,36.0717,-78.8235
7,Club Boulevard Elementary,"400 W Club Boulevard, Durham, NC",22.0,KG-5,36.0192,-78.8966
8,Durham School of the Arts,"401 North Duke Street, Durham, NC",75.0,6-12,36.0025,-78.9064
9,Durham's Performance Learning Center,"401 North Driver Street, Durham, NC",8.0,8-12,35.9896,-78.8795
10,E K Powe Elementary,"913 9th Street, Durham, NC",15.0,KG-5,36.0133,-78.9222
11,Easley Elementary,"302 Lebanon Circle, Durham, NC",74.0,KG-5,36.0754,-78.9269
12,Eastway Elementary,"610 North Alston Avenue, Durham, NC",11.0,KG-5,35.9912,-78.885


In [16]:
#Reset index
table_Durham.reset_index(drop = True, inplace = True)
table_Durham

,Name,Address,Rating,Grade span,Latitude,Longitude
0,Bethesda Elementary,"2009 S Miami Boulevard, Durham, NC",18.0,KG-5,35.9407,-78.8378
1,Brogden Middle,"1001 Leon Street, Durham, NC",37.0,6-8,36.0245,-78.9093
2,Central Park School For Child,"724 Foster Street, Durham, NC",74.0,KG-5,36.0046,-78.901
3,Chewning Middle,"5001 Red Mill Road, Durham, NC",21.0,8,36.0717,-78.8235
4,Club Boulevard Elementary,"400 W Club Boulevard, Durham, NC",22.0,KG-5,36.0192,-78.8966
5,Durham School of the Arts,"401 North Duke Street, Durham, NC",75.0,6-12,36.0025,-78.9064
6,Durham's Performance Learning Center,"401 North Driver Street, Durham, NC",8.0,8-12,35.9896,-78.8795
7,E K Powe Elementary,"913 9th Street, Durham, NC",15.0,KG-5,36.0133,-78.9222
8,Easley Elementary,"302 Lebanon Circle, Durham, NC",74.0,KG-5,36.0754,-78.9269
9,Eastway Elementary,"610 North Alston Avenue, Durham, NC",11.0,KG-5,35.9912,-78.885


In [17]:
#Convert Rating values to int.
table_Durham.Rating = table_Durham.Rating.astype(int)
type(table_Durham['Rating'][0])

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


numpy.int64

In [23]:
#Visualize schools on the map
map_Durham = folium.Map(location=[latitude, longitude], zoom_start=10.4)

#Set color scheme for the schools
i = 0
x = np.arange(100)
ys = [i + x + (i*x)**2 for i in range(100)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#Add markers to the map
markers_colors = []
for lat, lon, name, rate in zip(table_Durham['Latitude'], table_Durham['Longitude'], table_Durham['Name'], table_Durham['Rating']):
    label = folium.Popup(str(name) + ', Rate: ' + str(rate), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[rate-1],
        fill=True,
        fill_color=rainbow[rate-1],
        fill_opacity=0.7).add_to(map_Durham)
       
map_Durham

In [29]:
kmeans.labels_

NameError: name 'kmeans' is not defined

In [33]:
#Run k-means to cluster the schools into 5 clusters.

kclusters = 5 #Number of clusters
Durham_schools_clustering = table_Durham['Rating'].reshape(-1, 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Durham_schools_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


array([0, 3, 2, 0, 0, 2, 4, 0, 2, 4, 0, 1, 1, 0, 0, 0, 4, 1, 0, 2, 2, 3, 4,
       0, 0, 1, 0, 1, 3, 4, 0, 4, 0, 4], dtype=int32)

In [34]:
#Add cluster labels to the dataframe.
table_Durham.insert(0, 'Cluster Labels', kmeans.labels_)
table_Durham

,Cluster Labels,Name,Address,Rating,Grade span,Latitude,Longitude
0,0,Bethesda Elementary,"2009 S Miami Boulevard, Durham, NC",18,KG-5,35.9407,-78.8378
1,3,Brogden Middle,"1001 Leon Street, Durham, NC",37,6-8,36.0245,-78.9093
2,2,Central Park School For Child,"724 Foster Street, Durham, NC",74,KG-5,36.0046,-78.901
3,0,Chewning Middle,"5001 Red Mill Road, Durham, NC",21,8,36.0717,-78.8235
4,0,Club Boulevard Elementary,"400 W Club Boulevard, Durham, NC",22,KG-5,36.0192,-78.8966
5,2,Durham School of the Arts,"401 North Duke Street, Durham, NC",75,6-12,36.0025,-78.9064
6,4,Durham's Performance Learning Center,"401 North Driver Street, Durham, NC",8,8-12,35.9896,-78.8795
7,0,E K Powe Elementary,"913 9th Street, Durham, NC",15,KG-5,36.0133,-78.9222
8,2,Easley Elementary,"302 Lebanon Circle, Durham, NC",74,KG-5,36.0754,-78.9269
9,4,Eastway Elementary,"610 North Alston Avenue, Durham, NC",11,KG-5,35.9912,-78.885


In [37]:
table_Durham.sort_values(by=['Cluster Labels'])

,Cluster Labels,Name,Address,Rating,Grade span,Latitude,Longitude
0,0,Bethesda Elementary,"2009 S Miami Boulevard, Durham, NC",18,KG-5,35.9407,-78.8378
26,0,Research Triangle Charter,"2418 Ellis Road, Durham, NC",26,KG-8,35.9367,-78.8505
24,0,Northern High,"117 Tom Wilkinson Road, Durham, NC",23,9-12,36.0927,-78.9125
23,0,Neal Middle,"201 Baptist Road, Durham, NC",20,6-8,35.9799,-78.7794
18,0,Holt Elementary,"4019 Holt School Road, Durham, NC",21,KG-5,36.0562,-78.9065
32,0,WG Pearson Magnet Middle,"600 E Umstead St, Durham, NC",27,7-8,35.982,-78.8966
15,0,Global Scholars Academy,"311 Dowd Street, Durham, NC",16,KG-5,36.0006,-78.8928
13,0,George Watts Elementary,"700 Watts Street, Durham, NC",21,KG-5,36.0082,-78.9101
10,0,Eno Valley Elementary,"117 Milton Road, Durham, NC",25,KG-5,36.0854,-78.913
14,0,Glenn Elementary,"2415 E Geer Street, Durham, NC",23,KG-5,36.029,-78.8378


In [36]:
#Visualize clusters on the map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, name, cluster in zip(table_Durham['Latitude'], table_Durham['Longitude'], table_Durham['Name'], table_Durham['Cluster Labels']):
    label = folium.Popup(str(name) + ', Cluster: ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Explore venues nearby schools

In [38]:
# The code was removed by Watson Studio for sharing.

Foursquare ceredentials are defined but hidden


In [39]:
#Function that extracts the category of the venue.
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
#Function to explore all the schools.
def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['School Name', 
                  'School Latitude', 
                  'School Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
Durham_venues = getNearbyVenues(names=table_Durham['Name'],
                                   latitudes=table_Durham['Latitude'],
                                   longitudes=table_Durham['Longitude'])

Bethesda Elementary
Brogden Middle
Central Park School For Child
Chewning Middle
Club Boulevard Elementary
Durham School of the Arts
Durham's Performance Learning Center
E K Powe Elementary
Easley Elementary
Eastway Elementary
Eno Valley Elementary
Forest View Elementary
George L Carrington Middle
George Watts Elementary
Glenn Elementary
Global Scholars Academy
Healthy Start Academy
Hillandale Elementary
Holt Elementary
Little River Elementary
Mangum Elementary
Maureen Joy Charter School
Merrick-Moore Elementary
Neal Middle
Northern High
R N Harris Elementary
Research Triangle Charter
Riverside High
Sandy Ridge Elementary
Southern School of Energy and Sustainabi
Southern School of Engineering
The Wright School
WG Pearson Magnet Middle
Y E Smith Elementary


In [42]:
Durham_venues.head()

,School Name,School Latitude,School Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bethesda Elementary,35.940717,-78.837788,Bojangles' Famous Chicken 'n Biscuits,35.947947,-78.838097,Fast Food Restaurant
1,Bethesda Elementary,35.940717,-78.837788,New Japan,35.945882,-78.838916,Japanese Restaurant
2,Bethesda Elementary,35.940717,-78.837788,Frisky Business Boutique,35.947876,-78.836343,Lingerie Store
3,Bethesda Elementary,35.940717,-78.837788,Bethesda Gym,35.942294,-78.839091,Basketball Court
4,Bethesda Elementary,35.940717,-78.837788,Mineral Springs Road,35.939252,-78.840496,Moving Target


In [43]:
#Check the number of returned venues per school.
Durham_venues.groupby('School Name').count()

,School Latitude,School Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
School Name,,,,,,
Bethesda Elementary,13,13,13,13,13,13
Brogden Middle,60,60,60,60,60,60
Central Park School For Child,98,98,98,98,98,98
Chewning Middle,1,1,1,1,1,1
Club Boulevard Elementary,29,29,29,29,29,29
Durham School of the Arts,100,100,100,100,100,100
Durham's Performance Learning Center,21,21,21,21,21,21
E K Powe Elementary,77,77,77,77,77,77
Easley Elementary,4,4,4,4,4,4


In [44]:
#Number of unique categories in all returned venues.
print('There are {} uniques categories.'.format(len(Durham_venues['Venue Category'].unique())))

There are 182 uniques categories.


### Analyze each school

In [45]:
#One hot encoding
Durham_onehot = pd.get_dummies(Durham_venues[['Venue Category']], prefix="", prefix_sep="")

#Add school column back to dataframe
Durham_onehot['School Name'] = Durham_venues['School Name'] 

#Move school column to the first column
fixed_columns = [Durham_onehot.columns[-1]] + list(Durham_onehot.columns[:-1])
Durham_onehot = Durham_onehot[fixed_columns]

Durham_onehot.head()

,School Name,ATM,Accessories Store,African Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Bethesda Elementary,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bethesda Elementary,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bethesda Elementary,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bethesda Elementary,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bethesda Elementary,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
Durham_onehot.shape

(817, 183)

In [47]:
#Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Durham_grouped = Durham_onehot.groupby('School Name').mean().reset_index()
Durham_grouped

,School Name,ATM,Accessories Store,African Restaurant,American Restaurant,Arepa Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Bethesda Elementary,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Brogden Middle,0.000000,0.016667,0.000000,0.033333,0.000000,0.000000,0.016667,0.000000,0.000000,...,0.000000,0.016667,0.016667,0.000000,0.016667,0.000000,0.000000,0.016667,0.000000,0.000000
2,Central Park School For Child,0.000000,0.000000,0.000000,0.030612,0.000000,0.010204,0.010204,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010204,0.000000,0.000000,0.010204,0.000000
3,Chewning Middle,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Club Boulevard Elementary,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Durham School of the Arts,0.000000,0.000000,0.000000,0.030000,0.000000,0.010000,0.010000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000
6,Durham's Performance Learning Center,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000
7,E K Powe Elementary,0.000000,0.000000,0.012987,0.000000,0.012987,0.000000,0.012987,0.000000,0.000000,...,0.012987,0.000000,0.012987,0.012987,0.000000,0.000000,0.012987,0.000000,0.025974,0.000000
8,Easley Elementary,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Eastway Elementary,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000


In [48]:
Durham_grouped.shape

(32, 183)

In [50]:
#Print each school along with the top 5 most common venues.
num_top_venues = 5

for hood in Durham_grouped['School Name']:
    print("----"+hood+"----")
    temp = Durham_grouped[Durham_grouped['School Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bethesda Elementary----
                  venue  freq
0  Fast Food Restaurant  0.38
1   Fried Chicken Joint  0.08
2   Japanese Restaurant  0.08
3      Basketball Court  0.08
4        Lingerie Store  0.08


----Brogden Middle----
                 venue  freq
0       Clothing Store  0.07
1          Pizza Place  0.05
2           Shoe Store  0.05
3        Jewelry Store  0.03
4  American Restaurant  0.03


----Central Park School For Child----
                 venue  freq
0                  Bar  0.08
1                Hotel  0.04
2   Italian Restaurant  0.03
3          Pizza Place  0.03
4  American Restaurant  0.03


----Chewning Middle----
                 venue  freq
0               Garden   1.0
1                  ATM   0.0
2               Office   0.0
3  Monument / Landmark   0.0
4        Moving Target   0.0


----Club Boulevard Elementary----
                  venue  freq
0     Convenience Store  0.10
1  Fast Food Restaurant  0.07
2        Sandwich Place  0.07
3    Chinese Restaurant

In [51]:
#Function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
#Create the new dataframe and display the top 10 venues for each school.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['School']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
schools_venues_sorted = pd.DataFrame(columns=columns)
schools_venues_sorted['School'] = Durham_grouped['School Name']

for ind in np.arange(Durham_grouped.shape[0]):
    schools_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Durham_grouped.iloc[ind, :], num_top_venues)

schools_venues_sorted

,School,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bethesda Elementary,Fast Food Restaurant,Pawn Shop,Moving Target,Lingerie Store,Basketball Court,Fried Chicken Joint,Park,Japanese Restaurant,Construction & Landscaping,Diner
1,Brogden Middle,Clothing Store,Pizza Place,Shoe Store,Rental Car Location,Lingerie Store,Jewelry Store,Pharmacy,Gym / Fitness Center,Park,American Restaurant
2,Central Park School For Child,Bar,Hotel,Pizza Place,Gastropub,American Restaurant,Mexican Restaurant,Italian Restaurant,Brewery,Pub,Deli / Bodega
3,Chewning Middle,Garden,Zoo Exhibit,Ethiopian Restaurant,Flower Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Farm,Fabric Shop,Exhibit
4,Club Boulevard Elementary,Convenience Store,Sandwich Place,Park,Fast Food Restaurant,Chinese Restaurant,Peruvian Restaurant,Theme Park Ride / Attraction,Big Box Store,Dog Run,Food Truck
5,Durham School of the Arts,Bar,Hotel,Pizza Place,Brewery,American Restaurant,Mexican Restaurant,Gastropub,Italian Restaurant,Seafood Restaurant,Bakery
6,Durham's Performance Learning Center,Convenience Store,Fast Food Restaurant,Grocery Store,Gas Station,Hardware Store,Bakery,Park,Discount Store,Café,Food
7,E K Powe Elementary,Café,Record Shop,Bakery,Gastropub,Sandwich Place,Pizza Place,Shipping Store,Bookstore,Yoga Studio,Ice Cream Shop
8,Easley Elementary,Office,Candy Store,Intersection,Trail,Fast Food Restaurant,Farmers Market,Farm,Fabric Shop,Exhibit,Event Space
9,Eastway Elementary,Coffee Shop,Brewery,Building,Café,Food Truck,Garden,Spa,Gas Station,Tattoo Parlor,Park


In [54]:
schools_venues_sorted['Rating'] = table_Durham['Rating']
schools_venues_sorted

,School,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Rating
0,Bethesda Elementary,Fast Food Restaurant,Pawn Shop,Moving Target,Lingerie Store,Basketball Court,Fried Chicken Joint,Park,Japanese Restaurant,Construction & Landscaping,Diner,18
1,Brogden Middle,Clothing Store,Pizza Place,Shoe Store,Rental Car Location,Lingerie Store,Jewelry Store,Pharmacy,Gym / Fitness Center,Park,American Restaurant,37
2,Central Park School For Child,Bar,Hotel,Pizza Place,Gastropub,American Restaurant,Mexican Restaurant,Italian Restaurant,Brewery,Pub,Deli / Bodega,74
3,Chewning Middle,Garden,Zoo Exhibit,Ethiopian Restaurant,Flower Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Farm,Fabric Shop,Exhibit,21
4,Club Boulevard Elementary,Convenience Store,Sandwich Place,Park,Fast Food Restaurant,Chinese Restaurant,Peruvian Restaurant,Theme Park Ride / Attraction,Big Box Store,Dog Run,Food Truck,22
5,Durham School of the Arts,Bar,Hotel,Pizza Place,Brewery,American Restaurant,Mexican Restaurant,Gastropub,Italian Restaurant,Seafood Restaurant,Bakery,75
6,Durham's Performance Learning Center,Convenience Store,Fast Food Restaurant,Grocery Store,Gas Station,Hardware Store,Bakery,Park,Discount Store,Café,Food,8
7,E K Powe Elementary,Café,Record Shop,Bakery,Gastropub,Sandwich Place,Pizza Place,Shipping Store,Bookstore,Yoga Studio,Ice Cream Shop,15
8,Easley Elementary,Office,Candy Store,Intersection,Trail,Fast Food Restaurant,Farmers Market,Farm,Fabric Shop,Exhibit,Event Space,74
9,Eastway Elementary,Coffee Shop,Brewery,Building,Café,Food Truck,Garden,Spa,Gas Station,Tattoo Parlor,Park,11


In [56]:
schools_venues_sorted = schools_venues_sorted[['School', 'Rating', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue']]
schools_venues_sorted

,School,Rating,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bethesda Elementary,18,Fast Food Restaurant,Pawn Shop,Moving Target,Lingerie Store,Basketball Court,Fried Chicken Joint,Park,Japanese Restaurant,Construction & Landscaping,Diner
1,Brogden Middle,37,Clothing Store,Pizza Place,Shoe Store,Rental Car Location,Lingerie Store,Jewelry Store,Pharmacy,Gym / Fitness Center,Park,American Restaurant
2,Central Park School For Child,74,Bar,Hotel,Pizza Place,Gastropub,American Restaurant,Mexican Restaurant,Italian Restaurant,Brewery,Pub,Deli / Bodega
3,Chewning Middle,21,Garden,Zoo Exhibit,Ethiopian Restaurant,Flower Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Farm,Fabric Shop,Exhibit
4,Club Boulevard Elementary,22,Convenience Store,Sandwich Place,Park,Fast Food Restaurant,Chinese Restaurant,Peruvian Restaurant,Theme Park Ride / Attraction,Big Box Store,Dog Run,Food Truck
5,Durham School of the Arts,75,Bar,Hotel,Pizza Place,Brewery,American Restaurant,Mexican Restaurant,Gastropub,Italian Restaurant,Seafood Restaurant,Bakery
6,Durham's Performance Learning Center,8,Convenience Store,Fast Food Restaurant,Grocery Store,Gas Station,Hardware Store,Bakery,Park,Discount Store,Café,Food
7,E K Powe Elementary,15,Café,Record Shop,Bakery,Gastropub,Sandwich Place,Pizza Place,Shipping Store,Bookstore,Yoga Studio,Ice Cream Shop
8,Easley Elementary,74,Office,Candy Store,Intersection,Trail,Fast Food Restaurant,Farmers Market,Farm,Fabric Shop,Exhibit,Event Space
9,Eastway Elementary,11,Coffee Shop,Brewery,Building,Café,Food Truck,Garden,Spa,Gas Station,Tattoo Parlor,Park


### Cluster schools based on their ranking and nearby venues

In [57]:
#Run k-means to cluster the schools into 5 clusters.
Durham_grouped['Rating'] = table_Durham['Rating']
kclusters = 5 #Number of clusters
Durham_grouped_clustering = Durham_grouped.drop('School Name', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Durham_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 2, 4, 1, 1, 4, 3, 1, 4, 3, 1, 0, 0, 1, 1, 1, 3, 0, 1, 4, 4, 2, 3,
       1, 1, 0, 1, 0, 2, 3, 1, 3], dtype=int32)

In [58]:
#Add cluster labels to the dataframe.
schools_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
schools_venues_sorted

,Cluster Labels,School,Rating,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Bethesda Elementary,18,Fast Food Restaurant,Pawn Shop,Moving Target,Lingerie Store,Basketball Court,Fried Chicken Joint,Park,Japanese Restaurant,Construction & Landscaping,Diner
1,2,Brogden Middle,37,Clothing Store,Pizza Place,Shoe Store,Rental Car Location,Lingerie Store,Jewelry Store,Pharmacy,Gym / Fitness Center,Park,American Restaurant
2,4,Central Park School For Child,74,Bar,Hotel,Pizza Place,Gastropub,American Restaurant,Mexican Restaurant,Italian Restaurant,Brewery,Pub,Deli / Bodega
3,1,Chewning Middle,21,Garden,Zoo Exhibit,Ethiopian Restaurant,Flower Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Farm,Fabric Shop,Exhibit
4,1,Club Boulevard Elementary,22,Convenience Store,Sandwich Place,Park,Fast Food Restaurant,Chinese Restaurant,Peruvian Restaurant,Theme Park Ride / Attraction,Big Box Store,Dog Run,Food Truck
5,4,Durham School of the Arts,75,Bar,Hotel,Pizza Place,Brewery,American Restaurant,Mexican Restaurant,Gastropub,Italian Restaurant,Seafood Restaurant,Bakery
6,3,Durham's Performance Learning Center,8,Convenience Store,Fast Food Restaurant,Grocery Store,Gas Station,Hardware Store,Bakery,Park,Discount Store,Café,Food
7,1,E K Powe Elementary,15,Café,Record Shop,Bakery,Gastropub,Sandwich Place,Pizza Place,Shipping Store,Bookstore,Yoga Studio,Ice Cream Shop
8,4,Easley Elementary,74,Office,Candy Store,Intersection,Trail,Fast Food Restaurant,Farmers Market,Farm,Fabric Shop,Exhibit,Event Space
9,3,Eastway Elementary,11,Coffee Shop,Brewery,Building,Café,Food Truck,Garden,Spa,Gas Station,Tattoo Parlor,Park


In [59]:
#Prepare final table as the report.
schools_venues_sorted['Address'] = table_Durham['Address']
schools_venues_sorted['Grade Span'] = table_Durham['Grade span']
schools_venues_sorted = schools_venues_sorted[['School', 'Grade Span', 'Address', 'Rating', 'Cluster Labels', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue']]
schools_venues_sorted

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,School,Grade Span,Address,Rating,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bethesda Elementary,KG-5,"2009 S Miami Boulevard, Durham, NC",18,1,Fast Food Restaurant,Pawn Shop,Moving Target,Lingerie Store,Basketball Court,Fried Chicken Joint,Park,Japanese Restaurant,Construction & Landscaping,Diner
1,Brogden Middle,6-8,"1001 Leon Street, Durham, NC",37,2,Clothing Store,Pizza Place,Shoe Store,Rental Car Location,Lingerie Store,Jewelry Store,Pharmacy,Gym / Fitness Center,Park,American Restaurant
2,Central Park School For Child,KG-5,"724 Foster Street, Durham, NC",74,4,Bar,Hotel,Pizza Place,Gastropub,American Restaurant,Mexican Restaurant,Italian Restaurant,Brewery,Pub,Deli / Bodega
3,Chewning Middle,8,"5001 Red Mill Road, Durham, NC",21,1,Garden,Zoo Exhibit,Ethiopian Restaurant,Flower Shop,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Farm,Fabric Shop,Exhibit
4,Club Boulevard Elementary,KG-5,"400 W Club Boulevard, Durham, NC",22,1,Convenience Store,Sandwich Place,Park,Fast Food Restaurant,Chinese Restaurant,Peruvian Restaurant,Theme Park Ride / Attraction,Big Box Store,Dog Run,Food Truck
5,Durham School of the Arts,6-12,"401 North Duke Street, Durham, NC",75,4,Bar,Hotel,Pizza Place,Brewery,American Restaurant,Mexican Restaurant,Gastropub,Italian Restaurant,Seafood Restaurant,Bakery
6,Durham's Performance Learning Center,8-12,"401 North Driver Street, Durham, NC",8,3,Convenience Store,Fast Food Restaurant,Grocery Store,Gas Station,Hardware Store,Bakery,Park,Discount Store,Café,Food
7,E K Powe Elementary,KG-5,"913 9th Street, Durham, NC",15,1,Café,Record Shop,Bakery,Gastropub,Sandwich Place,Pizza Place,Shipping Store,Bookstore,Yoga Studio,Ice Cream Shop
8,Easley Elementary,KG-5,"302 Lebanon Circle, Durham, NC",74,4,Office,Candy Store,Intersection,Trail,Fast Food Restaurant,Farmers Market,Farm,Fabric Shop,Exhibit,Event Space
9,Eastway Elementary,KG-5,"610 North Alston Avenue, Durham, NC",11,3,Coffee Shop,Brewery,Building,Café,Food Truck,Garden,Spa,Gas Station,Tattoo Parlor,Park


In [63]:
#Visualize clusters on the map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, name, rate, cluster in zip(table_Durham['Latitude'], table_Durham['Longitude'], table_Durham['Name'], table_Durham['Rating'], schools_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(name) + ', Rating: ' + str(rate) + ', Cluster: ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters